In [4]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import numpy as np

from typing import Optional

from sklearn.base import BaseEstimator, TransformerMixin

In [38]:
# Loading data
train_df = pd.read_csv("./data/titanic_train.csv")
test_df = pd.read_csv("./data/titanic_test.csv")

In [17]:
class FillMissingAgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        self._mean_age: float = None
        self._mean_mrs_age: float = None
        self._mean_miss_age: float = None

    def _extract_title_from_name(self, full_name: str) -> pd.DataFrame:
        return full_name.split(",")[1].split(".")[0].strip()

    def fit(self, X: pd.DataFrame, y: Optional[np.ndarray] = None):
        data = X.copy()
        data["title"] = data["Name"].apply(self._extract_title_from_name)

        self._mean_age = data["Age"].mean()
        self._mean_mrs_age = data.loc[data["title"] == "mrs"]["Age"].mean()
        self._mean_miss_age = data.loc[data["title"] == "miss"]["Age"].mean()

        return self

    def transform(self, X: pd.DataFrame, y: Optional[np.ndarray] = None) -> pd.DataFrame:
        data = X.copy()
        data["title"] = data["Name"].apply(self._extract_title_from_name)

        data.loc[(data["title"] == "mrs"), "Age"] = data.loc[(data["title"] == "mrs"), "Age"].fillna(self._mean_mrs_age)
        data.loc[(data["title"] == "miss"), "Age"] = data.loc[(data["title"] == "miss"), "Age"].fillna(self._mean_miss_age)
        data["Age"].fillna(self._mean_age, inplace=True)
        
        return data

In [26]:
class AgeCategorisationTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X: pd.DataFrame, y: Optional[np.ndarray] = None):
        return self

    def _assign_age_group(self, age: float) -> str:
        if age <= 14:
            return "child"
        elif age <= 55:
            return "adult"
        return "elder"

    def transform(self, X: pd.DataFrame, y: Optional[np.ndarray] = None) -> pd.DataFrame:
        data = X.copy()
        data["age_group"] = data["Age"].apply(self._assign_age_group)
        
        return data

In [39]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [37]:
categorical_features = ["Pclass", "Sex", "Embarked", "age_group"]

pipeline = Pipeline([
    ("missing_age_filler", FillMissingAgeTransformer()),
    ("age_categoriser", AgeCategorisationTransformer()),
    ("one_hot_encoder", ColumnTransformer([("oh", OneHotEncoder(), categorical_features), ("cont_transformer")]))
])

In [40]:
pipeline.fit_transform(train_df)

array([[0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.]])